In [1]:
import pymc as pm
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
from scipy.stats import norm

In [32]:
# Patient Arrival Rate

arrivalRateMax = 322

# Ambulance Arrival Rate

ambulanceRateMax = 82

# Nurse Arrival Rate

arrivalRateNurseMax = 48

In [34]:
def modelWithDecisions(wfp, hoa, ro, draws):
  model = pm.Model()

  with model:
      # Decision Nodes
      workForcePlan = wfp
      hireOvernightAnalyst = hoa
      requestOvertime = ro

      # Number of Nurses to Arrive Based on Decisions
      workForcePlanNumber = workForcePlan*30 + (1-workForcePlan)*24
      nurses = workForcePlanNumber + 0.3*requestOvertime*(workForcePlanNumber)

      # Discrete Non-Conjugate (state of nature) Distributions
      canadaDay = pm.Bernoulli("Canada Day", 0.5)
      heatWave = pm.Bernoulli("Heat Wave", 0.5)
      powerOutage = pm.Bernoulli("Power Outage", (0.033+(heatWave*0.0396)))

      # Percentage of Patients that will arrive based on the state of nature (if canadaDay happens, if there is a heatwave, if there is a power outage)
      numPatientsCanadaDay = 1.1*(canadaDay) + (1-canadaDay)
      numPatientsHeatWave = 1.11*(heatWave) + (1-heatWave)
      numPatientsPowerOutage = 1.05*(powerOutage) + (1-powerOutage)
      patientPercentage = numPatientsCanadaDay*numPatientsHeatWave*numPatientsPowerOutage

      # Continuous Conjugate Prior Distributions of Arrival Rates
      ambulanceArrivalRate = pm.Gamma("Ambulance Arrival Rate", alpha = patientPercentage*ambulanceRateMax, beta = 2)
      patientArrivalRate = pm.Gamma("Walk-in Arrival Rate", alpha = patientPercentage*arrivalRateMax, beta = 2)

      # Continuous Non-Conjugate Distributions of Patient Severity
      patientSeverityAmbulance = pm.Triangular("Ambulance Severity", lower = 1, c=2.7, upper=5)
      patientSeverityWalkin = pm.Triangular("Walk-in Severity", lower = 1, c=3.07, upper=5)

      # Percentage of Nurses that will arrive based on the state of nature (canada day occuring)
      nurseCanadaDay = 0.96*(canadaDay) + (1-canadaDay)

      # Continuous Conjugate Posterior Distributions of Total Arrivals
      # Impacted by overnight analyst providing an observation given that you hire one
      # Overnight analyst has a p = 0.8 of giving an average observation and p = 0.2 of giving an above average one for patient arrivals
      if hireOvernightAnalyst == 1:
        prediction = np.random.binomial(1, p = 0.8)
        if prediction == 1:
          numberOfPatientsAmbulance = pm.Poisson("Number of Ambulance Patients", (ambulanceArrivalRate), observed = 53)
          numberOfPatientsWalkin = pm.Poisson("Number of Walk-in Patients", (patientArrivalRate), observed = 209)
        else:
          numberOfPatientsAmbulance = pm.Poisson("Number of Ambulance Patients", (ambulanceArrivalRate), observed = 64)
          numberOfPatientsWalkin = pm.Poisson("Number of Walk-in Patients", (patientArrivalRate), observed = 266)
      else:
        numberOfPatientsAmbulance = pm.Poisson("Number of Ambulance Patients", (ambulanceArrivalRate))
        numberOfPatientsWalkin = pm.Poisson("Number of Walk-in Patients", (patientArrivalRate))

      # Continuous Non-Conjugate Distribution of Nurse Arrivals, dependent on the number of nurses that arrive based on workforce plan and overtime request
      numberOfNurses = pm.Triangular("Number of Nurses", lower = 18, c = 24*nurseCanadaDay, upper = nurses*nurseCanadaDay)


      # It should be noted that there are minimums and maximums in place for the patientsMissed. The maximum is there due to the possibility that serviceMean can be negative, given a lower than average number of patient arrivals
      # and a higher than average number of nurse arrivals, which can result in a negative mean, which is set to 0. The minimum is there as there is the potential, due to the poisson distribution, of there being greater than
      # 374 number of patients missed, however this is not possible as defined by the utility function (there are always 18 nurses in staff).
      serviceMean = (numberOfPatientsWalkin*(1+((3.07-patientSeverityWalkin)/3.07)))+(numberOfPatientsAmbulance*(1+((2.7-patientSeverityAmbulance)/2.7)))-numberOfNurses*8

      # Patients missed, which is dependent on patient severity and the number of patients arriving in total. The serviceMean variable is the complex equation used to determine the mean number of patients missed.
      patientsMissed = pm.math.minimum(pm.Poisson("Number of Patients Not Serviced", pm.math.maximum(serviceMean, 0)), 374)

      # Continuous Non-Conjugate Distribution of Time Spent by DM, the time spent distribution is dependent on the decisions the DM makes
      timeSpent = pm.Uniform("Time Spent", lower = (8 + (2*workForcePlan + requestOvertime - 2*hireOvernightAnalyst)), upper = 12)

      # Cost functions that comprise Uy, cost of nurses is dependent on workforce plan being in effect and the overtime amount, analyst cost being dependent on hiring an analyst and fine cost being dependent on patients missed.
      costOfNurses = requestOvertime*((workForcePlanNumber)*467 + (workForcePlanNumber*0.3)*233.5) + (1-requestOvertime)*(workForcePlanNumber*467)
      analystCost = 4500*hireOvernightAnalyst
      fineCost = patientsMissed*500

      # Total cost
      cost = costOfNurses + analystCost + fineCost

      # Utilities for time spent and cost
      Ux = (pm.math.exp(timeSpent)-162754.79)/(-162351.36)
      Uy = (pm.math.sqrt(cost)-455.644)/(-357.8)

      # Multi variate utilities
      Uxy = 0.4*Ux + 0.2*Uy+0.4*Ux*Uy

      # Utility function, deterministic
      Utility = pm.Deterministic("Utility", Uxy)

      # Model running code, the draws is defined in the inputs
      with model:
        trace = pm.sample(draws= draws,tune=1000,chains=2, random_seed=0)

      # Returns the mean utility, model and trace, which were used for preliminary analysis
      return (trace['posterior']['Utility']).to_numpy().mean(), model, trace


In [35]:
# Runs through each possible scenario, with 20000 runs, tune of 1000 and 2 chains
utilitiesList = []
for i in range(2):
  for j in range(2):
    for k in range(2):
      utilitiesList.append([[i,j,k], modelWithDecisions(i,j,k, 20000)[0]])

/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)
/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)
/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)
/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)


/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)
/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)
/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)
/usr/local/lib/python3.10/dist-packages/pytensor/tensor/rewriting/math.py:1173: RuntimeWarning: divide by zero encountered in double_scalars
  v = reduce(np.multiply, num, one) / reduce(np.multiply, denum, one)


In [36]:
# For printing utilities
for l in utilitiesList:
  print("---------------------")
  if l[0][0] == 0:
    print("The workforce plan is business as usual")
  if l[0][0] == 1:
    print("The workforce plan is to have high number of staff")
  if l[0][1] == 0:
    print("Overnight Analyst is not hired")
  if l[0][1] == 1:
    print("Overnight Analyst is hired")
  if l[0][2] == 0:
    print("Overtime is not requested")
  if l[0][2] == 1:
    print("Overtime is requested")
  print("Utility is:", round(l[1], 3))

---------------------
The workforce plan is business as usual
Overnight Analyst is not hired
Overtime is not requested
Utility is: 0.69
---------------------
The workforce plan is business as usual
Overnight Analyst is not hired
Overtime is requested
Utility is: 0.608
---------------------
The workforce plan is business as usual
Overnight Analyst is hired
Overtime is not requested
Utility is: 0.638
---------------------
The workforce plan is business as usual
Overnight Analyst is hired
Overtime is requested
Utility is: 0.679
---------------------
The workforce plan is to have high number of staff
Overnight Analyst is not hired
Overtime is not requested
Utility is: 0.575
---------------------
The workforce plan is to have high number of staff
Overnight Analyst is not hired
Overtime is requested
Utility is: 0.447
---------------------
The workforce plan is to have high number of staff
Overnight Analyst is hired
Overtime is not requested
Utility is: 0.616
---------------------
The workfor